### Measuring transaction costs and trading volume

In [ ]:
import sys
import numpy as np
from numpy import divide
from numpy.linalg import multi_dot as mdot
from numpy.linalg import inv
from numpy import dot
import matplotlib.pyplot as plt
import pandas as pd
import os
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects import IntVector
from rpy2.robjects import StrVector
pandas2ri.activate()
from multiprocessing import Pool
import datetime as dt
import yfinance



In [ ]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
asset_names = list(assets.values())
#spx = pd.read_csv('../data/spx.csv').stack().tolist()

In [ ]:

def sim_garch_model(tickers, len_out_of_sample=0, ugarch_model="sGARCH", garch_order=(1, 1)):
    """
    ugarch_model: One of "sGARCH", "gjrGARCH", not implemented: "eGARCH"
    garch_order: Default: (1, 1)
    """
    tickers = [ticker.replace("BZ=F", "BZF").replace("GC=F", "GCF") for ticker in tickers]
    assert (ugarch_model in ("sGARCH", "gjrGARCH"))
    tickers = StrVector(tickers)
    garch_order = IntVector(garch_order)
    # Define the R script and load the instance in Python
    r = ro.r
    r['source']("C:\\Users\\caspe\\Documents\\thesis\\backtesting\\sim_mgarch.R")
    # Load the function we have defined in R.
    Sim_mgarch = ro.globalenv['Sim_mgarch']
    # Fit the MGARCH model and receive the result
    ugarch_dist_model = "std"       # t-distribution for the individual models
    coef, returns, sigmas = Sim_mgarch(tickers, len_out_of_sample, ugarch_model, ugarch_dist_model, garch_order)
    return coef, returns, sigmas


In [ ]:
coef, returns, sigmas=sim_garch_model(tickers, len_out_of_sample=0, ugarch_model="sGARCH", garch_order=(1, 1))

In [ ]:
plt.plot(returns)